In [802]:
#Import packages
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import shutil
import os
import time
import win32com.client

In [803]:
#Copperleaf username and password
username = 'vorbeckj'
password = '#'

In [804]:
#Copperleaf region codes
regions = ['bos','col','kedli','kedny','ri','ups']

In [805]:
#Read in list of investments to check by region
investments = pd.read_excel('C:/users/vorbej1/desktop/test.xlsx',sheet_name='Rhode Island', skiprows=3)

In [806]:
#Drop NAs - excel formatting issue
investments = investments.dropna(subset=['Investment Name']) 

In [807]:
#Initalize dataframe
backup_check = pd.DataFrame()
backup_check['Investments'] = investments['Investment Name']
backup_check['yes/no'] = ''

In [808]:
#Load Webdriver
driver = webdriver.Chrome('C:/Users/vorbej1/desktop/chromedriver.exe')

In [809]:
#Load C55
driver.get('https://ngg-prod.copperleafgroup.com/NGG_PROD/Copperleaf5/Pages/Home/Views/Home.aspx')

In [810]:
#Send username and password
driver.find_element_by_id('username').send_keys(username)
driver.find_element_by_id('password').send_keys(password)

In [811]:
#Submit
driver.find_element_by_xpath('//input[@tabindex="4"]').click()
time.sleep(5)

In [812]:
#Select investment search
driver.find_element_by_xpath('//a[@title="Search for investments using the advanced search features"]').click()
time.sleep(5)

In [813]:
#Remove filter if applied       
try:
    driver.find_element_by_class_name("cl-master__search-button--clear-filter").click()
except:
    pass

time.sleep(1.5)

In [814]:
#Filter to region in focus
driver.find_elements_by_xpath('//a[@tabindex="-1"]')[2].click()
time.sleep(2)

region_filter = driver.find_element_by_xpath('//li[@class="k-item k-filter-item k-state-default"]')
hover = ActionChains(driver).move_to_element(region_filter)
hover.perform()

filter_form = driver.find_element_by_xpath('//input[@title="Value"]')
time.sleep(1)

filter_form.send_keys(regions[4])
filter_form.send_keys(Keys.RETURN)

In [815]:
#For each investment in the excel file, search the database
#Depending on if investment has its backup data, assign yes/no
#Assign n/a to investments not in the database
#Write to csv for emailer

for i in backup_check.Investments:
    #print(i)
    joe = driver.find_element_by_class_name('k-input')
    joe.send_keys(i)
    joe.send_keys(Keys.RETURN)
    time.sleep(1.5)
    try:
        driver.find_element_by_link_text(i).click()
        time.sleep(2)
        driver.find_element_by_xpath('//span[@class="k-sprite k-icon cl-k__icon--attachment"]').click()
        html = driver.page_source
        if "Backup" in html:
            backup_check['yes/no'] = 'yes'
        else:
            backup_check['yes/no'] = 'no'
        driver.execute_script("window.history.go(-1)")
        time.sleep(1.5)
    except:
        time.sleep(1)
        backup_check['yes/no'] = 'n/a'
        joe.clear()

In [816]:
#Write dataframe to csv
backup_check.to_csv('C:/Users/vorbej1/desktop/backup_check_ri.csv', sep=',')

In [820]:
#Emailer
win32c = win32com.client.constants
app = win32com.client.Dispatch("Excel.Application")

outlook = win32com.client.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'joseph.vorbeck@nationalgrid.com'
mail.Subject = 'test'

attachment = mail.Attachments.Add('C:/Users/vorbej1/desktop/backup_check_ri.csv')

mail.HTMLBody = "\<html>\
<head>\
<meta http-equiv=""Content-Type"" content=""text/html; charset=iso-8859-1"">\
\
<style type=""text/css"">\
<!--\
\
body { font-size: 20pt font-family: sans-serif; }\
\
-->\
</style>\
</head>\
<body>\
      Hello\
</body>\
</html>"

mail.Send()

print('Email Complete')